In [13]:
import pandas as pd
import numpy as np

In [90]:
iowa = 'data/iowa_caucus_results.xlsx'
new_hampshire = 'data/new_hampshire_caucus_results.xlsx'
nevada = 'data/nevada_caucus_results.xlsx'
south_carolina = 'data/south_carolina_caucus_results.xlsx'


In [91]:
iowa = pd.read_excel(iowa)
new_hampshire = pd.read_excel(new_hampshire)
nevada = pd.read_excel(nevada)
south_carolina = pd.read_excel(south_carolina)

In [92]:
iowa.head()

,Counties,Sanders,Biden,Buttigieg,Warren,Klobuchar,Bennet,Bloomberg,Gabbard,Patrick,Steyer,Yang,Other,Uncommitted
0,Adair,102,39,71,31,86,0,0,0,0,16,55,0,0
1,Adams,51,77,51,43,77,0,0,0,0,0,0,0,0
2,Allamakee,109,101,249,54,140,0,0,0,0,8,39,0,0
3,Appanoose,104,176,232,8,48,0,0,0,0,24,8,0,0
4,Audubon,50,90,40,30,90,0,0,0,0,0,0,0,0


In [93]:
nevada.head()

,Counties,Sanders,Biden,Buttigieg,Warren,Klobuchar,Bennet,Delaney,Gabbard,Patrick,Steyer,Yang,Uncommitted
0,Carson City,96,10,39,19,33,0,0,0,0,19,0,0
1,Churchill,31,12,18,8,9,0,0,0,0,13,0,0
2,Clark,5293,2623,1297,917,230,0,0,3,0,442,0,2
3,Douglas,76,13,79,26,38,0,0,0,0,15,0,0
4,Elko,42,9,23,15,17,0,0,0,0,0,0,1


In [94]:
south_carolina.head()

,Counties,Biden,Sanders,Steyer,Buttigieg,Warren,Klobuchar,Yang,Harris,Booker,Gabbard,Castro
0,Abbeville,1129,286,312,80,60,42,6,0,6,26,0
1,Aiken,6765,3169,1986,1245,1030,607,39,0,20,194,0
2,Allendale,552,119,241,9,17,2,1,0,2,2,0
3,Anderson,5553,3121,1801,987,984,524,34,0,26,229,0
4,Bamberg,1099,277,387,26,43,19,2,0,5,4,0


In [134]:
columns = new_hampshire_df.columns
columns = columns.to_list()
columns

['Candidate',
 'Belknap',
 'Carroll',
 'Cheshire',
 'Coös',
 'Grafton',
 'Hillsborough',
 'Merrimack',
 'Rockingham',
 'Strafford',
 'Sullivan']

In [210]:
new_hampshire_df = new_hampshire.drop('State-wide', axis=1)
new_hampshire_df = new_hampshire_df.melt(id_vars=['Candidate'],value_vars=new_hampshire_df.columns.to_list()[1:-1])
new_hampshire_df = new_hampshire_df[~new_hampshire_df['value'].isin(['—'])]
new_hampshire_df['value'] = new_hampshire_df['value'].astype(int)
new_hampshire_df['State'] = "NH"
new_hampshire_df = new_hampshire_df[new_hampshire_df['Candidate'] != "Total"]

In [211]:
new_hampshire_df = new_hampshire_df.groupby(['variable','Candidate'])['value'].sum().reset_index()
# new_hampshire_df['Percentage'] = 100 * new_hampshire_df['value']  / new_hampshire_df['value'].sum()

In [208]:
new_hampshire_df

variable   Candidate                      
Belknap    Amy Klobuchar                      2383
           Andrew Yang                         248
           Ben Gleib (withdrawn)                 1
           Bernie Sanders                     2670
           Cory Booker (withdrawn)               8
           Deval Patrick                        35
           Elizabeth Warren                    839
           Henry Hewes                           2
           Jason Evritte Dunlap                  1
           Joe Biden                          1122
           Joe Sestak (withdrawn)                5
           John Delaney (withdrawn)              1
           Julián Castro (withdrawn)             2
           Kamala Harris (withdrawn)             6
           Lorenz Kraus                          3
           Marianne Williamson (withdrawn)       2
           Michael A. Ellinger                  38
           Michael Bennet                       29
           Mosie Boyd                  